In [ ]:
import gerrychain

from gerrychain import Graph, Partition, MarkovChain
from gerrychain.updaters import Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from gerrychain.constraints import contiguous

import networkx as nx
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from functools import partial

In [ ]:
graph = Graph.from_file("./data/GA_CD_example/")

In [ ]:
for key in graph.nodes[0].keys():
    print(key)

``` 
  {'APBVAP20': [0.529178733914834,
   0.5835856208817768,
   0.325959841129744,
   0.31219226754668045,
   0.6469278046382038,
   0.47365778499369376],
  'COLVAP20': [0.6284230784310049,
   0.6991969199718455,
   0.37949580759046775,
   0.5391578015119086,
   0.7317167909530444,
   0.5214507671895139],
  'HVAP20': [0.09924434451617083,
   0.11561129909006877,
   0.05353596646072374,
   0.2269655339652282,
   0.08478898631484061,
   0.0477929821958201],
  'WVAP20': [0.31253078959435293,
   0.2709610671551293,
   0.5776337020079435,
   0.3193458939133552,
   0.23671883962959928,
   0.44606898407993334],
  'effectiveness': 5,
  'step': 999}]
```

In [ ]:
cds = set({})
for node in graph.nodes:
    cds.add(graph.nodes[node]["CD"])

In [ ]:
cds

In [ ]:
initial_partition = Partition(
    graph,
    assignment="CD",
    updaters={
        "population": Tally("TOTPOP", alias="population"),
        "BVAP20": Tally("BVAP", alias="BVAP20"),
        "WVAP20": Tally("WVAP", alias="WVAP20"),
        "VAP20": Tally("VAP", alias="VAP20"),
    }
)

In [ ]:
lst = []
for key in initial_partition["BVAP20"].keys():
    lst.append(initial_partition["BVAP20"][key]/initial_partition["VAP20"][key])

lst.sort()
lst

In [ ]:
pop = sum(data["TOTPOP"] for node, data in initial_partition.graph.nodes(data=True))
bvap = sum(data["BVAP"] for node, data in initial_partition.graph.nodes(data=True))

In [ ]:
bvap 

In [ ]:
initial_partition

In [ ]:
proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target= pop / 14,
    epsilon=0.01,
    node_repeats=2, 
)


chain = MarkovChain(
    proposal=proposal,
    initial_state=initial_partition,
    constraints=[contiguous],
    accept=always_accept,
    total_steps=10000,
)

In [ ]:
def compute_percent_list(name, partition):
    keys = list(partition[name].keys())
    results = []
    for key in keys:
        results.append(partition[name][key] / partition["VAP20"][key])
    
    return {name: results}

In [ ]:
initial_partition.BVAP20

In [ ]:
results = []
for step, partition in enumerate(chain):
    if step > 8999:
        new_row = {"step": step - 8999}
        new_row.update(compute_percent_list("BVAP20", partition))
        new_row.update(compute_percent_list("WVAP20", partition))
        results.append(new_row)
    
    
    lst = [
        val / partition.VAP20[key]
        for key, val in partition.BVAP20.items() 
        if (val/partition.VAP20[key]) > 0.5]
    print(f"step: {step}, partition: {len(lst)}, lst: {lst}", end="\r")

results_df = pd.DataFrame(results)
results_df


In [ ]:
results

In [ ]:
import json

In [ ]:
with open("./data/ensemble_example.json", "w") as f:
    json.dump(results, f)